**Странный метод (с) построения TT-аппроксимации**

---

In [1]:
import numpy as np
import teneva
np.random.seed(42)

In [86]:
d = 5
n = 8

In [87]:
func = teneva.FuncDemoRosenbrock(d)
func.set_grid(n, kind='uni')
I_full = teneva.grid_flat(func.n)
Y_full = func.get_f_ind(I_full).reshape(func.n, order='F')

In [88]:
def f(i):
    return Y_full[tuple(i)]

In [89]:
m_trn = int(9.E+0)
I_trn = teneva.sample_lhs([n]*d, m_trn)
Y_trn = np.array([f(i) for i in I_trn])

In [90]:
m_tst = int(1.E+4)
I_tst = np.vstack([np.random.choice(k, m_tst) for k in [n]*d]).T
Y_tst = np.array([f(i) for i in I_tst])

In [91]:
def _pop_random(x):
    return x.pop(np.random.choice(range(len(x))))

def generate_indices(n, k_fix=None, k_ind=None):
    d = len(n)
    indices = [list(range(k)) for k in n]
    I = [[_pop_random(indices[i]) for i in range(d)] for _ in range(min(n))]
    I = np.array(I, dtype=int)
    if k_fix is not None and k_ind is not None:
        I[:, k_fix] = int(k_ind)
    return I


def build_random(f, n):
    d = len(n)
    Y = [np.ones([1, k, 1]) for k in n]
    I = generate_indices(n)
    for i in I:
        v = f(i)
        s = abs(v) / v if abs(v) > 1.E-16 else v
        v = abs(v)**(1./d) if abs(v) > 1.E-16 else 1.
        for k in range(d):
            Y[k][0, i[k], 0] = v * (1 if k < d-1 else s)
    return Y, I

def build_random_fix(f, n, k_fix=None, k_ind=None):
    d = len(n)
    Y = [np.ones([1, k, 1]) for k in n]
    I = generate_indices(n, k_fix, k_ind)
    for i in I:
        v = f(i)
        s = abs(v) / v if abs(v) > 1.E-16 else v
        v = abs(v)**(1./(d-1)) if abs(v) > 1.E-16 else 1.
        for k in range(d):
            if k != k_fix:
                Y[k][0, i[k], 0] = v * (1 if k < d-1 else s)
            else:
                Y[k][0, :, 0] = 0.
                Y[k][0, k_ind, 0] = 1.
    return Y, I

def build_boxes(f, n, k_fix):
    Y_all = []
    I_all = np.ones((0, len(n)), dtype=int)
    for k_ind in range(n[k_fix]):
        Y, I = build_random_fix(f, n, k_fix, k_ind)
        Y_all.append(Y)
        I_all = np.vstack((I_all, I))
    return np.array(Y_all), I_all

def join(Y_all):
    Y = teneva.copy(Y_all[0])
    for Y_cur in Y_all[1:]:
        Y = teneva.add(Y, Y_cur)
    return Y

In [92]:
Y_all = []
I_trn = np.zeros((0, d), dtype=int)
Y_trn = np.zeros(0, dtype=float)

for k_fix in [0]: # range(d):
    Y_all_cur, I_trn_cur = build_boxes(f, [n]*d, k_fix)
    Y_trn_cur = np.array([f(i) for i in I_trn_cur])
    Y_cur = join(Y_all_cur)
    
    print(f'--- Mode {k_fix:-2d}')
    r = teneva.erank(Y_cur)
    print(f'Rank      : {r:-7.2f}')
    err = teneva.accuracy_on_data(Y_cur, I_trn_cur, Y_trn_cur)
    print(f'Error trn : {err:-7.1e}')
    err = teneva.accuracy_on_data(Y_cur, I_tst, Y_tst)
    print(f'Error tst : {err:-7.1e}')
    
    Y_all.extend(Y_all_cur)
    I_trn = np.vstack((I_trn, I_trn_cur))
    Y_trn = np.hstack((Y_trn, Y_trn_cur))
    
Y = join(Y_all)

print(f'--- Result')
print(f'Size  trn : {I_trn.shape[0]:-7d}')
print(f'Size  tst : {I_tst.shape[0]:-7d}')
r = teneva.erank(Y_cur)
print(f'Rank      : {r:-7.2f}')
err = teneva.accuracy_on_data(Y_cur, I_trn, Y_trn)
print(f'Error trn : {err:-7.1e}')
err = teneva.accuracy_on_data(Y_cur, I_tst, Y_tst)
print(f'Error tst : {err:-7.1e}')

--- Mode  0
Rank      :    8.00
Error trn : 2.4e-16
Error tst : 4.8e-01
--- Result
Size  trn :      64
Size  tst :   10000
Rank      :    8.00
Error trn : 2.4e-16
Error tst : 4.8e-01
